In [1]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model

In [9]:
# Define number of classes
train_classes = ['Azli','FordPresident','Jiang','Kapoor','Obama']

In [2]:
# Define data paths and image size
train_data_dir = r'C:\Users\US593\OneDrive\Desktop\visual-kinship-identification-in-forensics\data\train'
validation_data_dir = r'C:\Users\US593\OneDrive\Desktop\visual-kinship-identification-in-forensics\data\val'
img_width, img_height = 224, 224  

In [3]:
# Load pre-trained VGG16 model (without top layers)
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))

In [4]:
# Freeze convolutional base layers (to prevent retraining these layers)
for layer in base_model.layers:
    layer.trainable = False

In [10]:
# Add new classification layers
x = base_model.output
x = Flatten()(x)
predictions = Dense(len(train_classes), activation='softmax')(x)  # Adjust output size for your classes

In [11]:
# Create final model
model = Model(inputs=base_model.input, outputs=predictions)

In [12]:
# Data augmentation for training data
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)

In [13]:
# Load training and validation data
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=32,  # Adjust batch size as needed
    class_mode='categorical'
)
validation_generator = ImageDataGenerator(rescale=1./255)
validation_generator = validation_generator.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=32,
    class_mode='categorical'
)

Found 988 images belonging to 5 classes.
Found 151 images belonging to 5 classes.


In [14]:
# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [15]:
# Train the model
model.fit(
    train_generator,
    epochs=10,  # Adjust number of epochs
    validation_data=validation_generator
)

Epoch 1/10
31/31 [==============================] - 41s 1s/step - loss: 0.8143 - accuracy: 0.8057 - val_loss: 0.5650 - val_accuracy: 0.7947
Epoch 2/10
31/31 [==============================] - 17s 549ms/step - loss: 0.2151 - accuracy: 0.9231 - val_loss: 0.2811 - val_accuracy: 0.9007
Epoch 3/10
31/31 [==============================] - 17s 553ms/step - loss: 0.1534 - accuracy: 0.9453 - val_loss: 0.3249 - val_accuracy: 0.8675
Epoch 4/10
31/31 [==============================] - 17s 557ms/step - loss: 0.1042 - accuracy: 0.9676 - val_loss: 0.3161 - val_accuracy: 0.8543
Epoch 5/10
31/31 [==============================] - 17s 553ms/step - loss: 0.0917 - accuracy: 0.9737 - val_loss: 0.1973 - val_accuracy: 0.9470
Epoch 6/10
31/31 [==============================] - 17s 546ms/step - loss: 0.0573 - accuracy: 0.9899 - val_loss: 0.2377 - val_accuracy: 0.9007
Epoch 7/10
31/31 [==============================] - 17s 548ms/step - loss: 0.0548 - accuracy: 0.9858 - val_loss: 0.1652 - val_accuracy: 0.9404
Ep

In [18]:
model.evaluate(validation_generator)

5/5 [==============================] - 2s 229ms/step - loss: 0.1991 - accuracy: 0.9338


[0.19911488890647888, 0.9337748289108276]

In [19]:
model.save("model.h5")